# Première itération avec poisson

année, loca, nb restau

In [ ]:
# ============================================================
# Trend par localité (Poisson + CAGR) à partir d'un CSV
# ============================================================
# Dépendances : pandas, numpy, statsmodels
# pip install pandas numpy statsmodels

import pandas as pd
import numpy as np
import statsmodels.api as sm
import warnings
from statsmodels.tools.sm_exceptions import PerfectSeparationError

# ---------- Paramètres ----------
CSV_PATH = "restaurant_count_by_locality.csv" 
YEAR_MIN, YEAR_MAX = 2011, 2025     
EXCLUDE_YEAR = []

# ---------- Chargement ----------
df = pd.read_csv(CSV_PATH)

# Normalisation de colonnes (au cas où)
df.columns = [c.strip().lower() for c in df.columns]
expected_cols = {"locality", "year", "num_restaurants"}
missing = expected_cols - set(df.columns)
if missing:
    raise ValueError(f"Colonnes manquantes dans le CSV: {missing}. "
                     f"Colonnes trouvées: {list(df.columns)}")

# Filtrage éventuel des années
mask = (df["year"].between(YEAR_MIN, YEAR_MAX)) & (~df["year"].isin(EXCLUDE_YEAR))
df = df.loc[mask].copy()

# Sécurité : on exclut lignes non positives (Poisson attend des comptes >= 0)
df = df[df["num_restaurants"] >= 0].copy()

# ---------- Exclure localités avec < 3 années distinctes ----------
year_counts = df.groupby("locality")["year"].nunique()
excluded_low_years = year_counts[year_counts < 3].index.tolist()

if excluded_low_years:
    print(f"⛔ Exclues (moins de 3 années): {len(excluded_low_years)} localités")

df_fit = df[~df["locality"].isin(excluded_low_years)].copy()

# Liste pour tracer les échecs de fit (par ex. singularités)
failed_fits = []

def poisson_trend_for_group(g: pd.DataFrame) -> pd.Series:
    """
    Calcule des indicateurs de tendance pour une localité donnée :
    - Annual Growth % (Poisson GLM): exp(beta_year) - 1
    - CAGR % entre la 1ère et la dernière année observées
    - Deviance explained (pseudo-R²)
    - p-value du coef de l'année
    - trend_score = (Annual Growth %) * max(0, deviance_explained)
    Hypothèse: g contient au moins 3 années distinctes (filtré en amont).
    """
    # Agrège si jamais il y avait des doublons (locality, year)
    g = (
        g.sort_values("year")
         .groupby(["locality", "year"], as_index=False)["num_restaurants"]
         .sum()
         .sort_values("year")
    )

    loc = g["locality"].iloc[0]

    # CAGR (simple, observé entre 1ère et dernière année)
    first_year = int(g["year"].iloc[0])
    last_year  = int(g["year"].iloc[-1])
    n_years = last_year - first_year
    first_val = float(g["num_restaurants"].iloc[0])
    last_val  = float(g["num_restaurants"].iloc[-1])
    if n_years > 0 and first_val > 0:
        cagr_pct = ((last_val / first_val) ** (1 / n_years) - 1.0) * 100.0
    else:
        cagr_pct = np.nan

    # Par défaut (au cas où le GLM échoue)
    annual_growth_pct = np.nan
    pval_year = np.nan
    dev_explained = np.nan
    trend_score = np.nan

    # Construction du design
    year_centered = g["year"] - g["year"].mean()
    X = pd.DataFrame({"const": 1.0, "year": year_centered.astype(float)})
    y = g["num_restaurants"].astype(float)

    # GLM Poisson (robuste aux petites déviations via HC3 si df_resid>0)
    model = sm.GLM(y, X, family=sm.families.Poisson())

    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            use_hc3 = (len(g) - X.shape[1]) > 0
            res = model.fit(cov_type="HC3") if use_hc3 else model.fit()

        beta_year = res.params.get("year", np.nan)
        pval_year = res.pvalues.get("year", np.nan) if hasattr(res, "pvalues") else np.nan
        if np.isfinite(beta_year):
            annual_growth_pct = (np.exp(beta_year) - 1.0) * 100.0

        # Pseudo-R² via deviance expliquée
        dev = getattr(res, "deviance", np.nan)
        null_dev = getattr(res, "null_deviance", np.nan)
        if np.isfinite(dev) and np.isfinite(null_dev) and null_dev > 0:
            dev_explained = max(0.0, 1.0 - (dev / null_dev))

        if np.isfinite(annual_growth_pct) and np.isfinite(dev_explained):
            trend_score = annual_growth_pct * dev_explained

    except (PerfectSeparationError, np.linalg.LinAlgError, ValueError) as e:
        failed_fits.append((loc, str(e)))

    return pd.Series({
        "locality": loc,
        "years_covered": f"{first_year}-{last_year}",
        "n_obs": len(g),
        "annual_growth_pct_poisson": annual_growth_pct,
        "cagr_pct": cagr_pct,
        "p_value_year": pval_year,
        "deviance_explained": dev_explained,
        "trend_score": trend_score
    })

# ---------- Calcul par localité ----------
trend_table = (
    df_fit.groupby("locality", group_keys=False)
          .apply(poisson_trend_for_group)
          .reset_index(drop=True)
)

# Tri par score décroissant (plus haut = plus dynamique)
trend_table_sorted = trend_table.sort_values("trend_score", ascending=False)

# Affichage et sauvegarde
pd.set_option("display.float_format", lambda x: f"{x:,.4f}")
display(trend_table_sorted)

OUT_CSV = "locality_trends_poisson.csv"
trend_table_sorted.to_csv(OUT_CSV, index=False)
print(f"📄 Résultats exportés → {OUT_CSV}")

# --------- (Optionnel) quelques lectures utiles ----------
# Filtre des tendances significatives au seuil 10%
signif_10 = trend_table_sorted[trend_table_sorted["p_value_year"] < 0.10]
print(f"\nLocalités avec tendance significative (p<0.10): {len(signif_10)}")
display(signif_10[["locality","annual_growth_pct_poisson","cagr_pct","p_value_year","trend_score"]].head(20))

# ---------- Journal des exclusions / échecs ----------
if excluded_low_years:
    print("\n⛔ Localités exclues (moins de 3 années distinctes):")
    print(", ".join(sorted(set(excluded_low_years))))

if failed_fits:
    print("\n⚠️  Localités non traitées (échec du fit Poisson) + message d'erreur:")
    for loc, msg in failed_fits[:50]:
        print(f"- {loc}: {msg}")
    if len(failed_fits) > 50:
        print(f"... (+{len(failed_fits)-50} supplémentaires)")


/home/vince/cours/301_ML/week6/models/EstimEat-backend/.venv/lib/python3.13/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/vince/cours/301_ML/week6/models/EstimEat-backend/.venv/lib/python3.13/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/vince/cours/301_ML/week6/models/EstimEat-backend/.venv/lib/python3.13/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/vince/cours/301_ML/week6/models/EstimEat-backend/.venv/lib/python3.13/site-packages/statsmodel

LinAlgError: Singular matrix